## LZ78 Python Interface for Denoising with Lookahead

### Setup
Remember to do `cd crates/python/ && maturin develop && cd ../..` from the `lz78_rust` directory after making any changes to the Rust or the Python interface!

In [1]:

import numpy as np
from sys import stdout
import numpy as np
import matplotlib.pyplot as plt

import scipy.signal as signal
from scipy.linalg import toeplitz
import dill as pickle
from lz78 import Sequence, LZ78Encoder, CharacterMap, BlockLZ78Encoder, LZ78SPA
from lz78 import encoded_sequence_from_bytes, spa_from_bytes

### Small Usage Example
This is equivalent to the traversal and lookahead tests in `spa.rs`.

In [2]:
spa = LZ78SPA(2, gamma=0.5)
input = Sequence([1, 0] * 1000, alphabet_size=2)
train_loss = spa.train_on_block(input) / 2000
train_loss

0.17043380775717953

In [3]:
pdf_traverse = spa.traverse_and_get_prob([0, 1])
pdf_traverse

[0.9883720930232558, 0.011627906976744186]

In [4]:
pdf_lookahead = spa.traverse_and_get_prob_with_lookahead(input=[1, 0, 1], lookahead=[1, 0])
pdf_lookahead

seed_data: [1, 0, 1, 0, 1, 0]
seed_data: [1, 0, 1, 1, 1, 0]


[1.0, 0.0]

In [5]:
def universal_denoiser_naive(signal_to_filter, spa_tree, char_map, window_len,pi_matrix, x_alphabet):
    sequence_l = len(signal_to_filter)
    
    Zt_index = char_map.str_to_symbol_list(signal_to_filter[window_len:sequence_l])
    filtered_signal = []
    for il in range(sequence_l-window_len):
        prob = spa_tree.traverse_and_get_prob(signal_to_filter[il:il+window_len])
        PX_t_Z_tm1_ele=np.matmul(np.linalg.inv(np.transpose(pi_matrix)),prob)
        alphabet_ind = Zt_index[il]
        num = pi_matrix[:,alphabet_ind] * PX_t_Z_tm1_ele
        den = prob[alphabet_ind]
        pxt = num/den
        pxt = pxt/np.sum(pxt)
        xt_hat = np.dot(pxt,x_alphabet)
        filtered_signal.append(xt_hat)
    
    return filtered_signal

def get_prob_from_MC(Dd, seed_str, spa_tree, char_map,num_exper):
    generated_seq = []
    for i_mc in range(num_exper):
        output, loss = spa_tree.generate_data(Dd, seed_data = seed_str,temperature =1, min_context= 3)
        generated_seq.append(output)
    # count the number of times of last element in the generated sequence
    prob = np.zeros(3)
    for i in range(num_exper):
        # charmap the generated sequence
        generated_ind = char_map.str_to_symbol_list(generated_seq[i][-1])
        #print(generated_ind)    
        prob[generated_ind] += 1
    prob = prob/num_exper
    return prob

def universal_denoiser_delay_mc(Delay,signal_to_filter_mc, spa_tree, char_map, 
                                window_len,pi_matrix, x_alphabet, num_exp):
    sequence_len = len(signal_to_filter_mc) 
    Zt_index = char_map.str_to_symbol_list(signal_to_filter_mc[window_len:sequence_len])
    filtered_signal_delay_mc = []
    for idmc in range(sequence_len-window_len):
        Zt_seed_str = signal_to_filter_mc[idmc:window_len+idmc-Delay]
        prob_mc = get_prob_from_MC(Delay, Zt_seed_str, spa_tree, num_exp)
        
        PX_t_Z_tm1_mc = np.matmul(np.linalg.inv(np.transpose(pi_matrix)),prob_mc)
        alphabet_ind = Zt_index[idmc]
        #print(alphabet_ind)
        num = pi_matrix[:,alphabet_ind] * PX_t_Z_tm1_mc
        den = prob_mc[alphabet_ind]
        pxt = num/den
        pxt = pxt/np.sum(pxt)
        xt_hat_mc = np.dot(pxt,x_alphabet)
        #print(xt_hat_mc)
        filtered_signal_delay_mc.append(xt_hat_mc)
    
    #print(filtered_signal_delay_mc)

    return filtered_signal_delay_mc

In [6]:
def universal_denoiser_lookahead(Lookahead,signal_to_filter_la, spa_tree, char_map: CharacterMap, 
                                window_len,pi_matrix, x_alphabet):

    sequence_len = len(signal_to_filter_la) 
    Zt_index =signal_to_filter_la[window_len:sequence_len]
    
    filtered_signal_lookahead = []
    #signal_to_filter_la = np.array(signal_to_filter_la)
    for ila in range(sequence_len-window_len):
        Zt_seed_str = signal_to_filter_la[ila:window_len+ila]
        Zt_lookahead_str = signal_to_filter_la[ila+window_len:ila+window_len+Lookahead]

        Zt_seed_vec = Zt_seed_str
        Zt_lookahead_vec = Zt_lookahead_str
        prob_la = spa_tree.traverse_and_get_prob_with_lookahead(Zt_seed_vec, Zt_lookahead_vec)
        
        PX_t_Z_tm1_la = np.matmul(np.linalg.inv(np.transpose(pi_matrix)),prob_la)
        alphabet_ind = Zt_index[ila]
        num = pi_matrix[:,alphabet_ind] * PX_t_Z_tm1_la
        den = prob_la[alphabet_ind]
        pxt = num/den
        pxt = pxt/np.sum(pxt)
        xt_hat_la = np.dot(pxt,x_alphabet)
        filtered_signal_lookahead.append(xt_hat_la)
    
    return filtered_signal_lookahead    

## Markov Example 

In [7]:
def generate_markov_sequence_random_initial(N, p):
    """
    Generates a Markov sequence of length N with two states: -1 and 1.
    The initial state is randomly chosen with a 50:50 chance.
    The probability of switching states is p, and the probability of staying in the same state is 1 - p.
    
    Parameters:
    N (int): Length of the Markov sequence.
    p (float): Probability of switching states.
    
    Returns:
    np.ndarray: The generated Markov sequence.
    """
    # Initialize the first state randomly with a 50:50 chance
    initial_state = np.random.choice([1, -1])
    
    # Initialize the sequence
    sequence = np.zeros(N)
    sequence[0] = initial_state

    # Generate the Markov sequence
    for i in range(1, N):
        if np.random.rand() < p:
            sequence[i] = -sequence[i-1]  # Switch state
        else:
            sequence[i] = sequence[i-1]  # Stay in the same state

    return sequence


In [8]:
## this initializes the sequence that trains the SPA
#N = 1_000_000_000   #--> between 3^18 and 3^19  
N =  100_000_00 
D = 2
p = 0.8 # probability of changing state
Xt= generate_markov_sequence_random_initial(N, p)
Nt = np.random.choice([-1, 1], size=N, p=[0.5, 0.5])
Zt=  Xt + Nt + 2
# convert Zt to integers
Zt = Zt.astype(int)
Zt_str = ''.join(str(num) for num in Zt)

In [9]:
stdout.flush() ## add for any long process
spa = LZ78SPA(3, gamma=0.5)
char_map =  CharacterMap(Zt_str)
input = Sequence(Zt_str, charmap =char_map)
train_loss = spa.train_on_block(input) / N
train_loss

1.469769495820801

In [10]:
pi_matrix_true = np.array([[0.5,0.5,0],[0.5,0,0.5],[0, 0.5,0.5]])
print(pi_matrix_true)


[[0.5 0.5 0. ]
 [0.5 0.  0.5]
 [0.  0.5 0.5]]


In [ ]:
len_alphabet = char_map.get_alphabet_size()
pi_matrix_approx = np.zeros((len_alphabet, len_alphabet))
# approximate the transition matrix from Xt to Zt
alphabet_Xt = [-1,0, 1]

def compute_pi_matrix(Xt,Zt_str,alphabet, alphabet_Xt):
    pi_matrix = np.ones((len(alphabet_Xt), len(alphabet)))
    for i in range(len(alphabet)):
        for j in range(len(alphabet)):
            # count the number of times alphabet[i] is followed by alphabet[j]
            count = 1
            for k in range(len(Xt)-1):
                if Xt[k] == alphabet_Xt[i] and Zt_str[k] == alphabet[j]:
                    count += 1
            pi_matrix[i,j] = count
    
    pi_matrix = pi_matrix / np.sum(pi_matrix, axis=1)[:,np.newaxis]
    return pi_matrix
pi_matrix_approx = compute_pi_matrix(Xt,Zt,char_map.get_alphabet())
pi_matrix_approx


In [11]:
stdout.flush()
LAH = 3 # lookahead
WL = 5 # window length
alphabet_Xt = [-1,0, 1]
Zt_to_filter = char_map.encode(Zt_str[1110:1120])
#Zt_encoded = char_map.encode("".join(Zt_to_filter))
Xt_hat = universal_denoiser_lookahead(LAH,Zt_to_filter, spa, char_map,WL, pi_matrix_true, alphabet_Xt)

pdfs: [39.0, 0.0]seed_data: [2, 1, 1, 0, 2, 0, 0, 1, 1]
seed_data: [2, 1, 1, 0, 2, 1, 0, 1, 1]
seed_data: [2, 1, 1, 0, 2, 2, 0, 1, 1]
pdfs: [46.0, 99.0, 53.0]seed_data: [1, 1, 0, 2, 0, 0, 1, 1, 2]
seed_data: [1, 1, 0, 2, 0, 1, 1, 1, 2]
seed_data: [1, 1, 0, 2, 0, 2, 1, 1, 2]
pdfs: [62.0, 202.0, 153.0]seed_data: [1, 0, 2, 0, 1, 0, 1, 2, 0]
seed_data: [1, 0, 2, 0, 1, 1, 1, 2, 0]
seed_data: [1, 0, 2, 0, 1, 2, 1, 2, 0]
pdfs: [79.0, 146.0, 90.0]seed_data: [0, 2, 0, 1, 1, 0, 2, 0]
seed_data: [0, 2, 0, 1, 1, 1, 2, 0]
seed_data: [0, 2, 0, 1, 1, 2, 2, 0]
pdfs: [252.0, 342.0, 89.0]seed_data: [2, 0, 1, 1, 2, 0, 0]
seed_data: [2, 0, 1, 1, 2, 1, 0]
seed_data: [2, 0, 1, 1, 2, 2, 0]


In [14]:
print(Xt_hat)

[-1.0, 0.45049504950495045, 0.07534246575342468, 1.0056179775280896, -0.9829787234042553]


In [16]:
X_clean = Xt[1110:1115] 
MSE_loss = np.mean((np.array(Xt_hat) - X_clean)**2)
MSE_loss

0.23145370635719567